# Acquire ACS Data Over Serial WITHOUT Knowledge From the Device File
For this example, you will need to connect an ACS to your computer via serial. 
The sensor should be on before running this notebook. Note that the ACS is in picoDOS for the first 10 seconds after it is powered on and may not register on a serial port.

Note: In this example, absorption and attenuation will look bad because the ACS is running in air.


In [1]:
from datetime import datetime, timezone

from acspype import ACSStream
from acspype.processing import convert_sn_str
import acspype.qaqc as acsqaqc
from acspype.utils import find_acs_port # This is a convenience function that finds the first port that has an ACS attached to it.

In [2]:
num_loops = 5  # Acquire X packets, and then stop. Otherwise, the while loop will run forever.

In [3]:
port = find_acs_port()  # Find the port that has an ACS attached to it.
with ACSStream(port) as acss: # Open a serial object.
    i = 0
    while True:  # Continuously read the stream until a full packet is found.
        acss.read_stream()
        acs_pkt = acss.find_packet()
        if acs_pkt.full_packet is not None:
            parsed_packet = acss.parse_packet(acs_packet = acs_pkt) # Parse the packet.
            
            # Run the gap and syntax tests.
            flag_gap = acsqaqc.gap_test(now = datetime.now(timezone.utc), 
                                        time_stmp = acs_pkt.daq_time, 
                                        buffer_length= len(acss._buffer), # The bytes in the serial buffer can be accessed with the _buffer attribute. This is really the only instance where you would need access to the buffer contents outside of the ACSStream class.
                                        record_length = parsed_packet.record_length)
            flag_syntax = acsqaqc.syntax_test(full_packet = acs_pkt.full_packet)
            
            # Print some of the results.
            print(f"---------- Packet {i} ----------")
            print(f"Acquisition Time: {parsed_packet.daq_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'}")
            print(f"Acquired Packet From: {convert_sn_str(parsed_packet.sn_int)}")
            print(f"Elapsed Time (ms): {parsed_packet.elapsed_time}")
            print(f"Absorption Signal Counts: {parsed_packet.a_signal}")
            if flag_gap == 1:
                print('Gap Test: Passed')
            elif flag_gap == 4:
                print('Gap Test: Failed')
            if flag_syntax == 1:
                print('Syntax Test: Passed')
            elif flag_syntax == 4:
                print('Syntax Test: Failed')
            print('\n')
            
            i += 1
            if i == num_loops:
                break

---------- Packet 0 ----------
Acquisition Time: 2025-04-30T21:37:46.685Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 4750556
Absorption Signal Counts: (454, 520, 604, 707, 824, 957, 1098, 1271, 1463, 1692, 1938, 2174, 2435, 2723, 3052, 3390, 3770, 4161, 4598, 5101, 5632, 6162, 6768, 7358, 7966, 8583, 9165, 9829, 10564, 11258, 11966, 12781, 13630, 14650, 15696, 16693, 17718, 18660, 19721, 20724, 21551, 22482, 23355, 24290, 25399, 26155, 28094, 28947, 29776, 30477, 31172, 31772, 32425, 33068, 33390, 33570, 33693, 33728, 33662, 33515, 33101, 32440, 31694, 30838, 30434, 30109, 29218, 28263, 27312, 26220, 25182, 24139, 22958, 21600, 20268, 19154, 18105, 16955, 15800, 14715, 13641, 12679, 11782, 10869)
Gap Test: Passed
Syntax Test: Passed


---------- Packet 1 ----------
Acquisition Time: 2025-04-30T21:37:46.941Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 4750807
Absorption Signal Counts: (449, 516, 603, 704, 824, 953, 1094, 1268, 1460, 1687, 1935, 2171, 2430, 2720, 3050, 338

In [4]:
acs_pkt

ACSPacket(daq_time=datetime.datetime(2025, 4, 30, 21, 37, 47, 691238, tzinfo=datetime.timezone.utc), full_packet=bytearray(b'\xff\x00\xff\x00\x02\xc0\x05\x01S\x00\x00\x0b\x01\xd0\x00\x00"\xadrc\xadA\x01\xe0\x02\xcc\x00H\x80\xc3\x01T\x02\r\x01\x93\x01\xf4\x01\xc3\x02U\x01\xd1\x02<\x02\x07\x02\xa9\x02\x19\x02\x87\x02[\x03\x0b\x02q\x02\xeb\x02\xbf\x03y\x02\xd6\x03[\x037\x03\xf7\x03C\x03\xdc\x03\xb9\x04s\x03\xb9\x04Y\x04G\x05\n\x04F\x04\xf2\x04\xf3\x05\xb1\x04\xe1\x05\x9e\x05\xb4\x06n\x05\x98\x06`\x06\x99\x07G\x06Y\x07E\x07\x8e\x08\x11\x07\x10\x08\x1f\x08z\x08\xe2\x07\xd7\x08\xf5\t}\t\xc4\x08\xb1\t\xe7\n\xa0\n\xc4\t\xa9\n\xf1\x0b\xea\x0b\xce\n\xa3\x0c\x15\r:\x0c\xea\x0b\xb9\r;\x0e\xb7\x0e\x1e\x0c\xd4\x0e\x8f\x10=\x0f[\x0e\r\x0f\xd8\x11\xf2\x10\xd0\x0fu\x11j\x13\xeb\x12`\x10\xec\x13\x1b\x15\xfc\x13\xe8\x12\\\x14\xcc\x18\x0e\x15\x8b\x14\x00\x16\x84\x1ao\x17E\x15\x92\x18y\x1c\xbb\x18\xee\x17+\x1aH\x1f\x19\x1a\xa9\x18\xcd\x1c-!\x86\x1c?\x1aI\x1d\xf9#\xc8\x1d\xe7\x1b\xfe\x1f\xb8&b\x1f\xd1\x1d\x